In [14]:
import pandas as pd
from web3 import Web3
import time

# Connect to Ganache
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

if web3.is_connected():
    print("✅ Connected to Ganache")
else:
    raise Exception("❌ Failed to connect to Ganache")

# Smart contract address (replace with your deployed one)
contract_address = "0x1f628b74E612D3450b1ec3DA353bB3e79183ae94"

# ABI (your full ABI pasted as Python list)
abi = [
    {
        "inputs": [],
        "stateMutability": "nonpayable",
        "type": "constructor"
    },
    {
        "anonymous": False,
        "inputs": [
            {"indexed": False, "internalType": "uint256", "name": "timestamp", "type": "uint256"},
            {"indexed": False, "internalType": "string", "name": "packageId", "type": "string"},
            {"indexed": False, "internalType": "string", "name": "category", "type": "string"},
            {"indexed": False, "internalType": "string", "name": "value", "type": "string"}
        ],
        "name": "DataStored",
        "type": "event"
    },
    {
        "inputs": [],
        "name": "MAX_ENTRIES",
        "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "name": "dataRecords",
        "outputs": [
            {"internalType": "uint256", "name": "timestamp", "type": "uint256"},
            {"internalType": "string", "name": "packageId", "type": "string"},
            {"internalType": "string", "name": "category", "type": "string"},
            {"internalType": "string", "name": "value", "type": "string"}
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [{"internalType": "uint256", "name": "index", "type": "uint256"}],
        "name": "getData",
        "outputs": [
            {"internalType": "uint256", "name": "timestamp", "type": "uint256"},
            {"internalType": "string", "name": "packageId", "type": "string"},
            {"internalType": "string", "name": "category", "type": "string"},
            {"internalType": "string", "name": "value", "type": "string"}
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "getDataCount",
        "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "owner",
        "outputs": [{"internalType": "address", "name": "", "type": "address"}],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {"internalType": "string", "name": "packageId", "type": "string"},
            {"internalType": "string", "name": "category", "type": "string"},
            {"internalType": "string", "name": "value", "type": "string"}
        ],
        "name": "storeData",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    }
]

# Load contract
contract = web3.eth.contract(address=contract_address, abi=abi)
web3.eth.default_account = web3.eth.accounts[0]

# Load the CSV
df = pd.read_csv("iot_data.csv")

# Upload selected fields: e.g., temperature, location, status
for _, row in df.iterrows():
    package_id = str(row['packageid'])
    
    entries = [
        ("Temperature", f"{row['temperature_C']}°C"),
        ("Location", f"{row['location_lat']},{row['location_long']}"),
        ("Status", str(row['status'])),
        ("Tamper", str(row['tamper_detected']))
    ]
    
    for category, value in entries:
        txn = contract.functions.storeData(package_id, category, value).transact({
            'from': web3.eth.default_account,
            'gas': 1000000
        })
        receipt = web3.eth.wait_for_transaction_receipt(txn)
        print(f"✅ Stored: {package_id} | {category} | {value} | TxHash: {receipt.transactionHash.hex()}")
        time.sleep(0.1)  # avoid flooding Ganache too fast

total_records = contract.functions.getTotalRecords().call()
print(f"Total Records: {total_records}")


record = contract.functions.getRecord(0).call()
print("First Stored Record:", record)


✅ Connected to Ganache
✅ Stored: PKG7795 | Temperature | 4.6°C | TxHash: 12972ca7240a5ebf6e3a6bf9f81c1265ae61cff1cb004357faa126d9e45f2a9d
✅ Stored: PKG7795 | Location | 10.118892,120.244188 | TxHash: 58218421053bdda830a3b3ea4976a481caa65e9c536bb61c3a80b8188ae64555
✅ Stored: PKG7795 | Status | Out for Delivery | TxHash: 3ce0b74b4e8eadd01f186bb5453cfd7d4a2783032247735dcd7a1db6145bdcf9
✅ Stored: PKG7795 | Tamper | False | TxHash: 5ee2602ab9cf2c83f3b546185b417a7cf95ddbe14e01356a57b0ef9b6d2e85d6
✅ Stored: PKG8284 | Temperature | 5.5°C | TxHash: 704fea7aab62d3f703b6990683672dbe415709af15c27deb879b8d0f80b457fa
✅ Stored: PKG8284 | Location | 13.208275,124.515339 | TxHash: 160ed2f4c724aa52137cb83edb880317016391d7bca9d08d31e89d4c08b60627
✅ Stored: PKG8284 | Status | In Transit | TxHash: dbd9ccda4cb260455d279dffb31c1305dbbcfc42ba92af86fa3d7a80a2b17cff
✅ Stored: PKG8284 | Tamper | False | TxHash: 3c4f2f5d06fd89bd285e5da6662e4ae8c96fae8be921083732e53a3cf3d23d92
✅ Stored: PKG1259 | Temperature | 5.1